In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('..')

In [3]:
from dataset.dataset.dsindex import FilesIndex
from preprocessing.ct_masked_batch import CTImagesMaskedBatch as CTIMB

### Global index

In [4]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [5]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [6]:
ind = FilesIndex(path=DIR_LUNA, no_ext=True)

### Create batch by subindexing

In [7]:
batch_masked = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))
batch_masked_2 = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))

### Test actions

* check load

In [8]:
batch_masked.load(fmt='raw')
batch_masked_2.load(fmt='raw')

In [9]:
batch_copy = deepcopy(batch_masked)

* load info about nodules in batches

In [10]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


In [11]:
batch_masked = batch_masked.fetch_nodules_info(nodules_df=nodules)
batch_masked_2 = batch_masked_2.fetch_nodules_info(nodules_df=nodules)

* create mask in one of two batches before resize

In [12]:
batch_masked.create_mask()

* check resize

In [13]:
batch_masked = batch_masked.resize(shape=(128, 256, 256), n_workers=6)

In [14]:
batch_masked_2 = batch_masked_2.resize(shape=(128, 256, 256), n_workers=6)

* create mask in the second batch after resize

In [15]:
batch_masked_2 = batch_masked_2.create_mask()

In [16]:
batch_masked_2.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'], 
      dtype='<U101')

In [17]:
batch_masked.indices

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.187108608022306504546286626125',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.161855583909753609742728521805'], 
      dtype='<U101')

### Compare

In [18]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (14, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        if batches[i][1] == 'scan':
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height)[0], **com_args)
        elif batches[i][1] == 'mask':
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height)[1], cmap=plt.cm.gray)
    plt.show()

In [19]:
interact(lambda height: plot_slices(height, 1, [batch_masked, 'scan'],
                                    [batch_masked, 'mask']
                                    ,[batch_masked_2, 'mask']
                                   ), height=(0.01, 0.99, 0.01))



TypeError: Invalid dimensions for image data

<function __main__.<lambda>>